# Imports

In [1]:
# Imports
import pandas as pd

import requests
import yaml
import json
import os
import firebase_admin
from datetime import datetime

#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

from dotenv import load_dotenv
 
load_dotenv(dotenv_path='/Users/daniel/OAN/credentials/contoan/.env')

shop_url = os.environ['SHOPIFY_ACCESS_URL']


## ACCESS FIREBASE

In [2]:
filename=os.environ['FIREBASE_FILENAME']

In [3]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## get transactions

In [2]:
r = requests.get(shop_url+"/shopify_payments/payouts.json",
                params={
                'limit':250,
                 })

In [3]:
#r = requests.get(shop_url+"/shopify_payments/balance/transactions.json",
#                params={
#                'limit':250,
#                 })

In [3]:
payouts = r.json()

In [4]:
len(payouts['payouts'])

187

#### filter by date

In [5]:
#filter by 2020
payouts2020 = []
for payout in payouts['payouts']:
    if payout['date'][0:4]=='2020':
        payouts2020.append(payout)

In [6]:
#date ='2020-09-29'
date ='2020-11-23'

In [7]:
#filter by date 2020-09-03
payouts2020 = []
for payout in payouts['payouts']:
    if datetime.fromisoformat(payout['date'])>datetime.fromisoformat(date):
        payouts2020.append(payout)

In [8]:
len(payouts2020)

24

In [9]:
payouts2020[6]

{'id': 61488562275,
 'status': 'paid',
 'date': '2020-12-28',
 'currency': 'EUR',
 'amount': '121.86',
 'summary': {'adjustments_fee_amount': '0.00',
  'adjustments_gross_amount': '0.00',
  'charges_fee_amount': '3.39',
  'charges_gross_amount': '125.25',
  'refunds_fee_amount': '0.00',
  'refunds_gross_amount': '0.00',
  'reserved_funds_fee_amount': '0.00',
  'reserved_funds_gross_amount': '0.00',
  'retried_payouts_fee_amount': '0.00',
  'retried_payouts_gross_amount': '0.00'}}

### get accounts info

In [26]:
col_query = db.collection('Collections_Info').stream()

In [27]:
col_info = []
for acc in col_query:
    print(u'{} => {}'.format(acc.id, acc.to_dict()))
    col_info.append(acc.to_dict())

Accounting => {'code': 2505}
Accounts => {'account_number': 572091}


## contabilizar

In [24]:
users = pd.read_gbq("""
SELECT *
FROM `oan-nikarit.contoan.users`
""")

Downloading: 100%|██████████| 31/31 [00:00<00:00, 40.35rows/s]


In [25]:
users[users['name'].str[0:2]=='Fi']

,id,name,last_name,email,cuenta
3,oXJJEfAEPxFYtdJ2pnaU,Firebase,Cloud,None,None


In [32]:
payouts2020

[{'id': 61803921507,
  'status': 'in_transit',
  'date': '2021-01-06',
  'currency': 'EUR',
  'amount': '445.42',
  'summary': {'adjustments_fee_amount': '0.00',
   'adjustments_gross_amount': '0.00',
   'charges_fee_amount': '12.38',
   'charges_gross_amount': '457.80',
   'refunds_fee_amount': '0.00',
   'refunds_gross_amount': '0.00',
   'reserved_funds_fee_amount': '0.00',
   'reserved_funds_gross_amount': '0.00',
   'retried_payouts_fee_amount': '0.00',
   'retried_payouts_gross_amount': '0.00'}},
 {'id': 61743956067,
  'status': 'paid',
  'date': '2021-01-05',
  'currency': 'EUR',
  'amount': '158.82',
  'summary': {'adjustments_fee_amount': '0.00',
   'adjustments_gross_amount': '0.00',
   'charges_fee_amount': '4.18',
   'charges_gross_amount': '163.00',
   'refunds_fee_amount': '0.00',
   'refunds_gross_amount': '0.00',
   'reserved_funds_fee_amount': '0.00',
   'reserved_funds_gross_amount': '0.00',
   'retried_payouts_fee_amount': '0.00',
   'retried_payouts_gross_amount': '

In [28]:
i=col_info[0]['code']+1
payouts2=[]
for payout in payouts2020[::-1]:
    d_conta={
       "creation_date":"2020-12-31T14:00:55.591Z",
       "execution_date":payout['date']+'T00:00:00.000Z',
       "place":'spain',
        #0DmODGTOEiM5lg9SGx0J - nikarit
        "project":'0DmODGTOEiM5lg9SGx0J',
        #w04441aFcU5b7pQm6Rd2 - españa
        "intervention":'w04441aFcU5b7pQm6Rd2',
        #no need for phase in movement
        "phase":None,
        # Dani - z5m936GA0t3vHM28QKhR
        # Firebase_bot - oXJJEfAEPxFYtdJ2pnaU
       "creator_user":"oXJJEfAEPxFYtdJ2pnaU",
        #Jose - IjBxXuBshlfq2MUzwBSK
        "user_in_charge":"IjBxXuBshlfq2MUzwBSK",
        "type":'movement',
        #cantidad de la transferencia
       "amount":float(payout['amount']),
        "vat":0,
        # iA9Pzv2CImjItzwCaQv0 - Banco Stripe
        "account_id":'iA9Pzv2CImjItzwCaQv0',
        # vJbbj1kPxkcdXJyBOf1l - Banco Caja Ingenieros
        "target_id":'vJbbj1kPxkcdXJyBOf1l',
       "concept":'payout shopify',
        "description":'payout de banco stripe a caja de ingenieros',
        "images":[],
        "code":i,
        "origin":'script_dani'
        
    }
   
    i=i+1
    
    doc_acc = db.collection(u'Accounting').add(d_conta)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    
    payouts2.append(d_conta)

In [29]:
len(payouts2)

24

In [30]:
payouts2[4]

{'creation_date': '2020-12-31T14:00:55.591Z',
 'execution_date': '2020-12-08T00:00:00.000Z',
 'place': 'spain',
 'project': '0DmODGTOEiM5lg9SGx0J',
 'intervention': 'w04441aFcU5b7pQm6Rd2',
 'phase': None,
 'creator_user': 'oXJJEfAEPxFYtdJ2pnaU',
 'user_in_charge': 'IjBxXuBshlfq2MUzwBSK',
 'type': 'movement',
 'amount': '116.97',
 'vat': 0,
 'account_id': 'iA9Pzv2CImjItzwCaQv0',
 'target_id': 'vJbbj1kPxkcdXJyBOf1l',
 'concept': 'payout shopify',
 'description': 'payout de banco stripe a caja de ingenieros',
 'images': [],
 'code': 2510,
 'origin': 'script_dani'}

In [31]:
db.collection(u'Collections_Info').document(u'Accounting').update({'code':i})

update_time {
  seconds: 1609887779
  nanos: 609631000
}

### contabilziar donaciones

In [40]:
meses = {
    0:'enero',
    1:'febrero',
    2:'marzo',
    3:'abril',
    4:'mayo',
    5:'junio',
    6:'julio',
    7:'agosto',
    8:'septiembre',
    9:'octubre'
}

In [41]:
donaciones=[]
#for p in range(8):
for p in [8]:
    d_conta={
       "creation_date":"2020-09-06T15:00:55.591Z",
       "execution_date":"2020-0"+str(p+1)+"-04T15:00:55.591Z",
       "place":'spain',
        #4zcptWXv2IqQFkIMz2MP - admin general
        "project":'4zcptWXv2IqQFkIMz2MP',
        #8DG4syjb1kUl8HzfEWz1 - españa
        "intervention":'8DG4syjb1kUl8HzfEWz1',
        "phase":None,
        # Dani - z5m936GA0t3vHM28QKhR
       "creator_user":"z5m936GA0t3vHM28QKhR",
        #Alex - uCF6bT532woZwE9vQOuv
        "user_in_charge":"uCF6bT532woZwE9vQOuv",
        "type":'income',
        #cantidad de la transferencia 19.51 alex 9.63 patricia y 9.46 antonio
       "amount":19.51,
        "vat":0,
        # I6vsoTqCKFAS1AK09qzW - Donaciones
        "account_id":'I6vsoTqCKFAS1AK09qzW',
        # vJbbj1kPxkcdXJyBOf1l - Banco Caja Ingenieros
        "target_id":'vJbbj1kPxkcdXJyBOf1l',
       "concept":'Donacion Alex del Estal '+meses[p]+' 2020',
        "description":'Donacion Alex del Estal '+meses[p]+' 2020',
        "images":[],
        "code":i,
        "origin":'script_dani'
        
    }
    d_conta2={
       "creation_date":"2020-09-06T15:00:55.591Z",
       "execution_date":"2020-0"+str(p+1)+"-16T15:00:55.591Z",
       "place":'spain',
        #4zcptWXv2IqQFkIMz2MP - admin general
        "project":'4zcptWXv2IqQFkIMz2MP',
        #8DG4syjb1kUl8HzfEWz1 - españa
        "intervention":'8DG4syjb1kUl8HzfEWz1',
        "phase":None,
        # Dani - z5m936GA0t3vHM28QKhR
       "creator_user":"z5m936GA0t3vHM28QKhR",
        #Alex - uCF6bT532woZwE9vQOuv
        "user_in_charge":"uCF6bT532woZwE9vQOuv",
        "type":'income',
        #cantidad de la transferencia 19.51 alex 9.63 patricia y 9.46 antonio
       "amount":9.63,
        "vat":0,
        # I6vsoTqCKFAS1AK09qzW - Donaciones
        "account_id":'I6vsoTqCKFAS1AK09qzW',
        # vJbbj1kPxkcdXJyBOf1l - Banco Caja Ingenieros
        "target_id":'vJbbj1kPxkcdXJyBOf1l',
       "concept":'Donacion Patricia Barcelo del Olmo '+meses[p]+' 2020',
        "description":'Donacion Patricia Barcelo del Olmo '+meses[p]+' 2020',
        "images":[],
        "code":i+1,
        "origin":'script_dani'
        
    }
    d_conta3={
       "creation_date":"2020-09-06T15:00:55.591Z",
       "execution_date":"2020-0"+str(p+1)+"-16T15:00:55.591Z",
       "place":'spain',
        #4zcptWXv2IqQFkIMz2MP - admin general
        "project":'4zcptWXv2IqQFkIMz2MP',
        #8DG4syjb1kUl8HzfEWz1 - españa
        "intervention":'8DG4syjb1kUl8HzfEWz1',
        "phase":None,
        # Dani - z5m936GA0t3vHM28QKhR
       "creator_user":"z5m936GA0t3vHM28QKhR",
        #Alex - uCF6bT532woZwE9vQOuv
        "user_in_charge":"uCF6bT532woZwE9vQOuv",
        "type":'income',
        #cantidad de la transferencia 19.51 alex 9.63 patricia y 9.46 antonio
       "amount":9.46,
        "vat":0,
        # I6vsoTqCKFAS1AK09qzW - Donaciones
        "account_id":'I6vsoTqCKFAS1AK09qzW',
        # yfy4cPxkmFwIYVnoWD0A - Paypal
        "target_id":'yfy4cPxkmFwIYVnoWD0A',
       "concept":'Donacion Antonio Jaime Azuara '+meses[p]+' 2020',
        "description":'Donacion Antonio Jaime Azuara '+meses[p]+' 2020',
        "images":[],
        "code":i+2,
        "origin":'script_dani'
        
    }
   
    i=i+3
    
    doc_acc = db.collection(u'Accounting').add(d_conta)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    doc_acc = db.collection(u'Accounting').add(d_conta2)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    doc_acc = db.collection(u'Accounting').add(d_conta3)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})
    
    
    donaciones.append(d_conta)
    donaciones.append(d_conta2)
    donaciones.append(d_conta3)

In [8]:
ventas = db.collection('Nikarit_Sales').stream()

In [9]:
d_ventas = []
for venta in ventas:
    d_ventas.append(venta.to_dict())

In [10]:
df_ventas = pd.DataFrame(d_ventas)

In [11]:
df_ventas[df_ventas['total_price'] == 278.30]

,product_movement,manteca_unit_price,order_id,crema_nb,packHidra_unit_price,packContigo_unit_price,taxes_included,gateway,client_last_name,zip,...,city,client_phone,navidadCode,bolsa_unit_price,packAbanico_total_price,packAbanico_nb,bolsa_total_price,donation,message,to


In [12]:
df_ventas["client_gender"].value_counts()

Unknown    809
Female      28
Male         7
Other        4
Name: client_gender, dtype: int64

## AÑADIR UNA VENTA NUEVA

In [38]:
#coger venta que nos interesa copiar - Jose Pedro venta pack Navidad
venta_ex = db.collection('Nikarit_Sales').document('Es21K7uWRNu17m0ShrAn').get().to_dict()

In [39]:
##le quito el pack navidad de la venta_ex
venta_ex['packNavidad_nb'] = 13
venta_ex['packNavidad_unit_price']

venta_ex['packNavidad_total_price'] = round(venta_ex['packNavidad_unit_price']*venta_ex['packNavidad_nb'],2)

#modifico el resto de campos
#productos vendidos
#venta_ex['manteca_nb'] = 5
#venta_ex['manteca_unit_price']= 5.0
#venta_ex['manteca_total_price'] = venta_ex['manteca_nb']*venta_ex['manteca_unit_price']

#datos de la venta
venta_ex['creation_date'] = "2020-12-16T11:21:16.836Z"
venta_ex['close_date'] = venta_ex['creation_date']

#datos cliente
venta_ex['client_last_name'] = "Jordan"
venta_ex['client_name'] = "Jose Manuel"
venta_ex['client_gender'] = "Male"
venta_ex['city']= "Valladolid"

#precio total
price=venta_ex['packNavidad_total_price']
venta_ex['total_price'] = price
venta_ex["subtotal_price"] = price
venta_ex['total_tax']= round(price*(1-1/1.21),2)

In [40]:
doc_acc = db.collection(u'Nikarit_Sales').add(venta_ex)[1]
doc_acc_id = doc_acc.id
doc_acc.update({'id':doc_acc_id})

update_time {
  seconds: 1610137105
  nanos: 329887000
}